In [58]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Dense, Concatenate, Flatten, Dropout,BatchNormalization
from keras.models import Model, Sequential
import keras
import matplotlib.pyplot as plt
import random
import numpy as np
import tensorflow as tf
# Read the CSV file
data = pd.read_csv("R.csv")

In [59]:

labels = data.copy()

feature_headings = [
    "d1","d2","d3","d4","d5","d6","First Layer","Second Layer"
]

output_values = labels[feature_headings]
input_features = labels[[c for c in labels.columns if c not in feature_headings]]
label_encoder = LabelEncoder()

output_values["First Layer"] = label_encoder.fit_transform(output_values["First Layer"])
output_values["Second Layer"] = label_encoder.transform(output_values["Second Layer"])

unique_materials = pd.unique(data[['First Layer', 'Second Layer']].values.ravel())


num_materials = len(unique_materials)
num_wavelengths = 351

input_train, input_test, output_train, output_test = train_test_split(input_features, output_values, test_size=0.2, random_state=42)
input_train, input_val, output_train, output_val = train_test_split(input_train, output_train, test_size=0.2, random_state=42)

/tmp/ipykernel_132296/1402317043.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_values["First Layer"] = label_encoder.fit_transform(output_values["First Layer"])
/tmp/ipykernel_132296/1402317043.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_values["Second Layer"] = label_encoder.transform(output_values["Second Layer"])


In [64]:
i = Input(num_wavelengths,)
dl1 = Dense(num_wavelengths, activation="PReLU", input_shape=(num_wavelengths,))(i)
dl2 = Dense(256, activation="PReLU")(dl1)
out1 = Dense(num_materials, "softmax", name="first_layer")(dl2)
out2 = Dense(num_materials, "softmax", name="second_layer")(dl2)

classifier = Model(inputs=i, outputs=[out1,out2])
classifier.compile(
    optimizer="adam",
    loss={
        "first_layer": keras.losses.SparseCategoricalCrossentropy(),
        "second_layer": keras.losses.SparseCategoricalCrossentropy(),
    },
    metrics={
        "first_layer": tf.metrics.SparseCategoricalAccuracy("acc1"),
        "second_layer": tf.metrics.SparseCategoricalAccuracy("acc2")
    }
)

In [65]:
classifier.fit(
    input_train,
    {
        "first_layer": output_train["First Layer"],
        "second_layer": output_train["Second Layer"]
    },
    epochs=100,
    validation_data=(
        input_val,
        {
            "first_layer": output_val["First Layer"],
            "second_layer": output_val["Second Layer"]
        }
    )
)

Epoch 1/100
600/600 [==============================] - 3s 4ms/step - loss: 6.2658 - first_layer_loss: 3.0359 - second_layer_loss: 3.2299 - first_layer_acc1: 0.1164 - second_layer_acc2: 0.0925 - val_loss: 5.9910 - val_first_layer_loss: 2.8726 - val_second_layer_loss: 3.1184 - val_first_layer_acc1: 0.1717 - val_second_layer_acc2: 0.1131
Epoch 2/100
600/600 [==============================] - 2s 4ms/step - loss: 5.7964 - first_layer_loss: 2.7326 - second_layer_loss: 3.0638 - first_layer_acc1: 0.2010 - second_layer_acc2: 0.1262 - val_loss: 5.6452 - val_first_layer_loss: 2.6198 - val_second_layer_loss: 3.0254 - val_first_layer_acc1: 0.2052 - val_second_layer_acc2: 0.1296
Epoch 3/100
600/600 [==============================] - 2s 3ms/step - loss: 5.5072 - first_layer_loss: 2.5346 - second_layer_loss: 2.9727 - first_layer_acc1: 0.2552 - second_layer_acc2: 0.1556 - val_loss: 5.4930 - val_first_layer_loss: 2.5541 - val_second_layer_loss: 2.9389 - val_first_layer_acc1: 0.2288 - val_second_layer_ac

In [ ]:
regression = Sequential([
    Dense(num_wavelengths),
    Dense(256),
    Dense(6)
])

regression.compile("adam", "mean_squared_error")
regression.fit(input_train,output_train[["d1","d2","d3","d4","d5","d6"]], epochs=10, validation_data=(input_val, output_val[["d1","d2","d3","d4","d5","d6"]]))

Epoch 1/10
600/600 [==============================] - 2s 3ms/step - loss: 394.3272 - val_loss: 297.0521
Epoch 2/10
600/600 [==============================] - 2s 3ms/step - loss: 299.6229 - val_loss: 290.0248
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 296.7748 - val_loss: 312.8666
Epoch 4/10
600/600 [==============================] - 2s 3ms/step - loss: 298.7979 - val_loss: 290.2840
Epoch 5/10
600/600 [==============================] - 2s 3ms/step - loss: 294.7226 - val_loss: 291.8187
Epoch 6/10
600/600 [==============================] - 2s 3ms/step - loss: 296.3443 - val_loss: 289.7348
Epoch 7/10
600/600 [==============================] - 2s 3ms/step - loss: 296.1261 - val_loss: 291.3210
Epoch 8/10
600/600 [==============================] - 2s 3ms/step - loss: 295.6977 - val_loss: 291.1700
Epoch 9/10
600/600 [==============================] - 2s 3ms/step - loss: 295.0799 - val_loss: 291.7741
Epoch 10/10
600/600 [==============================] - 2s 3ms/st